# I. Data Exploration 
We are going to start with some basic data exploration to see what is in the Dataframes.
Specifically, we are looking for outliers and missing values and keeping an eye open for 
other unexpected qualities of the data. 

By the time we merge the frames and conclude the preprocessing of our dataset, we want to 
be completely familiar with the contents to avoid any surprizes in the next stages.

Next we inspect the correlation and statistical properties of the dataset and apply some dimensionality reduction and clustering to the dataset to see if we can group our subjects into subsets. 

In [43]:
# Basic tools
import os
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path

# Env vars from setup for inports
load_dotenv()

True

## 1. Basics

Woookay lets go over the individual frames real quick to see whats in there. 

### 1.1 Age and Gender
This dataframe contains age and gender data, attributed to individual, pseudonomized subject, related to via the primary 'id' key.

From the following analysis we obtain:
- id: counter primary key 
- gender: 
    - binary male-female
    - close to balanced but slightly in favor of male population
    - no missing values
- age: 
    - only major up to age 85 (18 jahre ist man erwachsen in deutschland nicht 21)
    - no missing values

In [67]:
# Data file paths
data_dir = os.getenv("DATA")
age_gender_df = pd.read_csv(Path(data_dir) / "alter_geschlecht.csv")
age_gender_df = age_gender_df.sort_values(by=["id"])
age_gender_df.head()

,id,Geschlecht,Alter
302985,1,Male,44
416478,2,Male,76
132773,3,Male,47
52197,4,Male,21
35343,5,Female,29


In [68]:
# We could technically use the .describe(include='all') method to get all the stats at once
# but we want that sweet fine grained control allowing us to know EVERYTHING

# Age
print(f"""Alter:
- max: {age_gender_df['Alter'].max()}
- min: {age_gender_df['Alter'].min()}
- mean: {age_gender_df['Alter'].mean()}
- median: {age_gender_df['Alter'].median()}
- std: {age_gender_df['Alter'].std()}
- count: {age_gender_df['Alter'].count()}
- nancount: {age_gender_df['Alter'].isna().sum()}
""")

# Gender
print(f"""Geschlecht:
- unique: {age_gender_df['Geschlecht'].unique()}
- count: {age_gender_df['Geschlecht'].count()}
- male count: {age_gender_df['Geschlecht'].value_counts()['Male']}
- female count: {age_gender_df['Geschlecht'].value_counts()['Female']}
""")

# ID
print(f"""id:
- min: {age_gender_df['id'].min()}
- max: {age_gender_df['id'].max()}
- nunique: {age_gender_df['id'].nunique()}
- count: {age_gender_df['id'].count()}
""")

Alter:
- max: 85
- min: 20
- mean: 38.8084133300272
- median: 36.0
- std: 15.500178638692834
- count: 508146
- nancount: 0

Geschlecht:
- unique: ['Male' 'Female']
- count: 508146
- male count: 274325
- female count: 233821

id:
- min: 1
- max: 508146
- nunique: 508146
- count: 508146



### 1.2 Interest Data (Target)
This is our target dataset, containing information on wether the subject is interested in the insurance or not. From the following analysis we can take away:

- id:
    - again our primary key
    - weirdly this counts up to only 381109
- interest:
    - binary 0 or 1
    - 7 times more 0 than 1s
    - no missing values

In [19]:
interest_df = pd.read_csv(Path(data_dir) / "interesse.csv")
interest_df = interest_df.sort_values(by=["id"])
interest_df.head()

,id,Interesse
0,1,1.0
1,2,0.0
2,3,1.0
3,4,0.0
4,5,0.0


In [40]:
# We could technically use the .describe(include='all') method to get all the stats at once
# but we want that sweet fine grained control allowing us to know EVERYTHING

# Interest
print(f"""Interesse:
- unique: {interest_df['Interesse'].unique()}  
- count: {interest_df['Interesse'].count()}
- 1 count: {interest_df['Interesse'].value_counts()[1]}
- 0 count: {interest_df['Interesse'].value_counts()[0]}
- nancount: {interest_df['Interesse'].isna().sum()}
""")

# ID
print(f"""id:
- min: {interest_df['id'].min()}
- max: {interest_df['id'].max()}
- nunique: {interest_df['id'].nunique()}
- count: {interest_df['id'].count()}
""")

Interesse:
- unique: [1. 0.]  
- count: 381109
- 1 count: 46710
- 0 count: 334399
- nancount: 0

id:
- min: 1
- max: 381109
- nunique: 381109
- count: 381109



### 1.3 Insurance Features
This Dataframe has a bit more content. That means we are going to use the big stick and slap it with .describe.
We quickly notice that most features are categorical with less than 4 categories, which is good for one-hot encoding.
Regional code and sales channels are categorical but ordingal encoding, even though not appropriate will have to do.

- id:
    - full range counter primary key again

- Driver's License
    - binary 0-1
    - mostly 1

- Regional code:
    - 53 different codes

- Previous Insurance:
    - binary 0-1
    - close to being balanced

- Vehicle Age:
    - most vehicles are 1< x <2 years old with very few older than two years

- sales channel:
    - 157 different channels

In [58]:
insurance_features_df = pd.read_csv(Path(data_dir) / "rest.csv", delimiter=";")
insurance_features_df = insurance_features_df.sort_values(by=["id"])

dtype_mapping = {
    "Fahrerlaubnis": "category", 
    "Vorversicherung": "category", 
    "Alter_Fzg": "category", 
    "Vorschaden": "category", 
    "Vertriebskanal": "category"
}
insurance_features_df = insurance_features_df.astype(dtype_mapping)
insurance_features_df["Regional_Code"] = insurance_features_df["Regional_Code"].astype(int).astype("category"
                                                                                                   )
insurance_features_df.head()

,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,id
163016,1,28,0,> 2 Years,Yes,40454.0,26.0,217,1
375123,1,3,0,1-2 Year,No,33536.0,26.0,183,2
56906,1,28,0,> 2 Years,Yes,38294.0,26.0,27,3
352098,1,11,1,< 1 Year,No,28619.0,152.0,203,4
122431,1,41,1,< 1 Year,No,27496.0,152.0,39,5


In [59]:
insurance_features_df.describe()

,Jahresbeitrag,Kundentreue,id
count,508146.000000,508146.000000,508146.000000
mean,30554.453041,154.340123,254073.500000
std,17146.574625,83.668793,146689.259281
min,2630.000000,10.000000,1.000000
25%,24381.000000,82.000000,127037.250000
50%,31661.000000,154.000000,254073.500000
75%,39403.750000,227.000000,381109.750000
max,540165.000000,299.000000,508146.000000


In [61]:
insurance_features_df.describe(include='category')

,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Vertriebskanal
count,508146,508146,508146,508146,508146,508146.0
unique,2,53,2,3,2,157.0
top,1,28,0,1-2 Year,Yes,152.0
freq,507097,141937,275076,267015,256248,179523.0


In [52]:
print(f"""Alter_Fzg:
- unique: {insurance_features_df['Alter_Fzg'].unique()}
- < 1 Year count: {insurance_features_df['Alter_Fzg'].value_counts()['< 1 Year']}
- > 2 Years count: {insurance_features_df['Alter_Fzg'].value_counts()['> 2 Years']}
""")

print(f"""Vorschaden:
- unique: {insurance_features_df['Vorschaden'].unique()}
- No count: {insurance_features_df['Vorschaden'].value_counts()['No']}
""")

print(f"""Total nans:
- total nan count: {insurance_features_df.isna().sum().sum()}      
""")

Alter_Fzg:
- unique: ['1-2 Year' '< 1 Year' '> 2 Years']
- < 1 Year count: 219805
- > 2 Years count: 21326

Vorschaden:
- unique: ['No' 'Yes']
- No count: 251898

Total nans:
- total nan count: 0      



## 2. Corelation & Clustering

Lets look at some intrinsic features of the data. First we'll merge then we'll cluster the data. Merging will reduce the set size, since somehoooow the target is the frame with missing entry, but this can't stop us so lets go.

In [73]:
# Throwing it together (may the lord come get me if this is wrong and I'm messing up already)
dataset_df = pd.merge(left=age_gender_df, right=interest_df, on="id")
dataset_df = pd.merge(left=dataset_df, right=insurance_features_df, on="id")
dataset_df.head()

,id,Geschlecht,Alter,Interesse,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue
0,1,Male,44,1.0,1,28,0,> 2 Years,Yes,40454.0,26.0,217
1,2,Male,76,0.0,1,3,0,1-2 Year,No,33536.0,26.0,183
2,3,Male,47,1.0,1,28,0,> 2 Years,Yes,38294.0,26.0,27
3,4,Male,21,0.0,1,11,1,< 1 Year,No,28619.0,152.0,203
4,5,Female,29,0.0,1,41,1,< 1 Year,No,27496.0,152.0,39


## Corelation

Nexxxt we are rolling out that sweet correlation tile map. But first we will have to make everything floatable, meaning that all the categoricals will be ordinated.

In [74]:
dataset_df.corr()

ValueError: could not convert string to float: 'Male'